## SET UP

In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
from glob import glob
from scipy.io import loadmat
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot
from datetime import datetime
from IPython.display import clear_output
import os, sys, shutil
from tqdm import tqdm, trange
from glob import glob
import cv2 as cv
import torch.nn.functional as F
import json
from sklearn.preprocessing import LabelEncoder

In [ ]:
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
# os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

In [ ]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/ECGDataDenoised"
label_file = main_dir + "/Diagnostics.xlsx"

In [ ]:
diag_df = pd.read_excel(label_file)
label_df = diag_df[['FileName', 'Rhythm']]
label_df

In [ ]:
data_paths = []
for file in glob(data_dir +"/*"):
    data_paths.append(file)

In [ ]:
label_encoder = LabelEncoder()
label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])
print(label_df)

In [ ]:
filenames = label_df["FileName"].values.tolist()

In [ ]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

In [ ]:
data_paths = []
for file in glob(data_dir +"/*"):
    data_paths.append(file)
print(len(data_paths))

In [ ]:
ratio = [0.8, 0.1]

train_index = int(len(data_paths)*ratio[0])
valid_index = int(len(data_paths)*(ratio[0]+ratio[1]))
print(train_index)
train_mat_paths = data_paths[:train_index]
valid_mat_paths = data_paths[train_index:valid_index]
print(len(train_mat_paths))

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]

## DATA LOADER

In [ ]:
class HeartData(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        # data = loadmat(data_path)['ECG'][0][0][2]
        data = pd.read_csv(data_path, header = None)
        data = data.values.T
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["FileName"] == filename]["Rhythm"].values.item()

        return clip_data, label

    def __len__(self):
        return len(self.data_paths)    

In [ ]:
train_ds = HeartData(train_mat_paths, label_df)
valid_ds = HeartData(valid_mat_paths, label_df)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 0)
batch_size = 6

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    # shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

## MODEL

In [ ]:
class Dilatex12(nn.Module):
    def __init__(self, in_channel):
        super(Dilatex12, self).__init__()
        self.in_channel = in_channel

        #Dilation block
        self.dilate1 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=1, padding = 0) # 296
        self.dilate2 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=2, padding = 2) 
        self.dilate3 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=3, padding= 4) 
        self.dilate4 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=4, padding= 6) 

        self.conv1 = nn.Sequential(
            nn.BatchNorm1d(24),
            nn.ReLU(),
            nn.MaxPool1d(3, stride = 2)
        )
                
        # FIRST Conv1D layer (on the left branch)
        self.conv1d_left = nn.Conv1d(24, out_channels=16, kernel_size=6, stride=1)
        
        # FIRST Conv1D layer (right branch)
        self.conv1d_right_1 = nn.Sequential(
            nn.Conv1d(24, out_channels=8, kernel_size=3, stride=1), 
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1),
        )
        self.conv1d_right_2 = nn.Sequential(
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1),
            nn.BatchNorm1d(16)
        )
        self.add = nn.Sequential(
            nn.ReLU(),
            nn.MaxPool1d(2, stride = 1)
        )

        # SECOND Conv1D (left)
        self.conv1d_left2 = nn.Conv1d(16, out_channels=16, kernel_size=7, stride=1)
        
        # SECOND Conv1D layer (right)
        self.conv1d_right2_1 = nn.Sequential(
            nn.Conv1d(16, out_channels=16, kernel_size=3, stride=1), 
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1),
        )
        self.conv1d_right2_2 = nn.Sequential(
            nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, stride=1),
            nn.BatchNorm1d(16)
        )

        # 2 LAYERS BI-LSTM
        self.lstm1 = nn.LSTM(16, 12, bidirectional=True, num_layers=2, batch_first=True)
        self.tanh = nn.Tanh() 
        self.batch1 = nn.BatchNorm1d(24)
        self.drop1 = nn.Dropout(0.01)
        self.lstm2 = nn.LSTM(24, 24, bidirectional=True, num_layers=2, batch_first=True)
        self.batch2 = nn.BatchNorm1d(48)
        self.drop2 = nn.Dropout(0.02)

    def forward(self, x):
        out_list = []
        a = torch.tensor_split(x, 12, dim = 1)
        for x_lead in a:
            x5 = torch.cat((self.dilate1(x_lead),self.dilate2(x_lead),self.dilate3(x_lead),self.dilate4(x_lead)),1)
            # print(x5.size())
            out_block1 = self.conv1(x5)
            # print(out_block1.size())
            
            y1 = self.conv1d_left(out_block1)
            # print(y1.size())
            y2 = self.conv1d_right_1(out_block1)
            y2 = self.conv1d_right_2(y2)
            # print(y2.size())
            out_add1 = torch.add(y1,y2)
            # print(out_add1.size())
    
            z1 = self.conv1d_left2(out_add1)
            z2 = self.conv1d_right2_1(out_add1)
            z2 = self.conv1d_right2_2(z2)
            out_add2 = torch.add(z1,z2)
            # print(out_add2.size())
    
            out_add2 = out_add2.permute(0, 2, 1)
    
            lstm1, _ = self.lstm1(out_add2)
            lstm1 = self.tanh(lstm1)
            lstm1 = lstm1.permute(0, 2, 1)
            lstm1 = self.batch1(lstm1)
            lstm1 = lstm1.permute(0, 2, 1)
            lstm1 = self.drop1(lstm1)
            out, _ = self.lstm2(lstm1)
            out = self.tanh(out)
            out = out.permute(0, 2, 1)
            out = self.batch2(out)
            out = out.permute(0, 2, 1)
            out = self.drop2(out)
            out_list.append(out.detach().to(device))
        output = torch.cat(out_list,1) 
        return output

In [ ]:
model1 = Dilatex12(1)

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, feature_dim):
        super(AttentionLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(feature_dim, feature_dim))  # Weight matrix
        self.b = nn.Parameter(torch.zeros(1, 1, feature_dim))         # Bias vector

    def forward(self, H):
        Q = torch.tanh(torch.matmul(H, self.W) + self.b)  # Shape: (batch_size, sequence_length, feature_dim)
        A = torch.bmm(Q, H.transpose(1, 2))  # Shape: (batch_size, sequence_length, sequence_length)
        W_attn = F.softmax(A, dim=-1)  # Shape: (batch_size, sequence_length, sequence_length)
        Yattn = torch.bmm(W_attn, H)  
        return Yattn
        
model2 = AttentionLayer(feature_dim=48)

In [ ]:
import torch
import torch.nn as nn

class ClassificationNetwork(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationNetwork, self).__init__()
        self.conv2D = nn.Sequential(
            nn.Conv2d(1, out_channels=64, kernel_size=(1, 12), stride=(1, 1)), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.dense1 = nn.Sequential(
            nn.Linear(64, 128),
            nn.LayerNorm(128),  # Corrected to BatchNorm1d
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.dense2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.LayerNorm(64),  # Corrected to BatchNorm1d
            nn.ReLU(),
            nn.Dropout(0.15)
        )
        
        self.output_layer = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension, shape: [batch_size, 1, seq_len, features]
        x = self.conv2D(x)  # Output shape: [batch_size, 64, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten, shape: [batch_size, 64]
        
        # First Dense Block
        x = self.dense1(x)    
        # Second Dense Block
        x = self.dense2(x)
        
        # Output Layer
        x = self.output_layer(x)
        if self.output_layer.out_features == 1:
            x = torch.sigmoid(x)  # For binary classification
        else:
            x = torch.softmax(x, dim=1)  # For multi-class classification

        return x

In [ ]:
model = ClassificationNetwork(num_classes=11)

## TRAINING

In [ ]:
epoch = 150
lr = 0.001 # lr = 0.001 Acc: 0.821875 lr = 0.0005 Acc: 0.825 Focal_loss Acc: 0.828
best_acc = 0
best_ep = 0
model.to(device)
model1.to(device)
model2.to(device)
optimizer = Adam([
    {'params': model1.lstm1.parameters(), 'weight_decay': 0.009},
    {'params': model1.lstm2.parameters(), 'weight_decay': 0.01},
    {'params': model.dense1.parameters(), 'weight_decay': 0.005},
    {'params': model.dense2.parameters(), 'weight_decay': 0.009},
    {'params': model.output_layer.parameters(), 'weight_decay': 0.0}  # No regularization
], lr=0.001)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))

In [ ]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)
    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()
loss_fn = nn.CrossEntropyLoss()
checkpoint_folder = "run_attention_gamma0.3_0.01_lr0001"

In [ ]:
def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = save_dir + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

In [ ]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [ ]:
class_la = []
for i in range (11):
    class_la.append(i)
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23
for e in range(epoch):
    model1.train()
    model2.train()
    model.train()
    print(f"Epoch: {e+1}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        # print(train_sig.shape)
        train_label = train_label.to(device)

        out1 = model1(train_sig)
        out2 = model2(out1)
        pred = model(out2)

        loss = loss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
        
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    log_dict["train"]["loss"].append(total_loss)
    log_dict["train"]["acc"].append(correct)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")

# Valid
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model1.eval()
    model2.eval()
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_label = valid_label.to(device)
            
            valid_sig = valid_sig.to(device)  
            out1 = model1(valid_sig)
            out2 = model2(out1)
            pred = model(out2)
            pred_pos = pred.argmax(1)
            
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
            
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        log_dict["valid"]["loss"].append(val_total_loss)
        log_dict["valid"]["acc"].append(val_correct)

        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e 

        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
        if val_correct >= old_valid_class_acc:
            old_valid_class_acc = val_correct
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'test_acc': val_correct
            }
         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = save_dir + f"/{checkpoint_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            save_best_model_dir = run_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()

acc_loss_json(log_dict, check_folder = checkpoint_folder)
    
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred, pos_label = 0)

auc1 = metrics.auc(fpr, tpr)

reports = classification_report(y_true, pred, output_dict=True) 
classification_report_csv(report = reports, auc = auc1, check_folder = checkpoint_folder)